In [2]:
# ============================================
# INSTALACIÓN DE DEPENDENCIAS (COLAB)
# ============================================

print("📦 INSTALANDO DEPENDENCIAS...")
print("=" * 60)

# Estrategia: NumPy 2.x (compatible con los modelos guardados)
# En Colab o local, instalar MediaPipe y OpenCV
!pip install mediapipe==0.10.21 numpy==1.26.4 protobuf==4.25.8 --upgrade --force-reinstall
!pip install opencv-python gradio matplotlib pandas scikit-learn -q
print("✅ Dependencias instaladas")
print("⚠️  Ignorar warnings de compatibilidad NumPy/MediaPipe\n")

📦 INSTALANDO DEPENDENCIAS...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 8.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.1 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
^C
✅ Dependencias instaladas
⚠️  Ignorar warnings de compatibilidad NumPy/MediaPipe



In [1]:
# ============================================
# IMPORTS Y CONFIGURACIÓN
# ============================================

import gradio as gr
import cv2
import numpy as np
import mediapipe as mp
import joblib
import warnings
warnings.filterwarnings('ignore')

print(f"✅ MediaPipe: {mp.__version__}")
print(f"✅ NumPy: {np.__version__}")
print(f"✅ OpenCV: {cv2.__version__}")
print(f"✅ Gradio: {gr.__version__}\n")

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


✅ MediaPipe: 0.10.21
✅ NumPy: 1.26.4
✅ OpenCV: 4.11.0
✅ Gradio: 5.49.1



In [2]:
# ============================================
# SUBIR Y CARGAR MODELOS DESDE /content/
# ============================================

from google.colab import files
from pathlib import Path
import os

print("📤 SUBIR ARCHIVOS DE MODELO")
print("=" * 60)
print("\n⚠️  IMPORTANTE: Sube estos 4 archivos en orden:")
print("   1. randomforest_model.pkl")
print("   2. scaler.pkl")
print("   3. pca.pkl")
print("   4. label_encoder.pkl")
print("\n📁 Los archivos se guardarán en /content/\n")

# Subir archivos
uploaded = files.upload()

print(f"\n✅ Archivos subidos: {len(uploaded)}")
for filename in uploaded.keys():
    size_mb = len(uploaded[filename]) / (1024 * 1024)
    print(f"   📦 {filename} ({size_mb:.2f} MB)")

# Verificar que todos los archivos necesarios estén presentes
required_files = {
    'randomforest_model.pkl': 'Modelo Random Forest',
    'scaler.pkl': 'Scaler (normalización)',
    'pca.pkl': 'PCA (reducción dimensionalidad)',
    'label_encoder.pkl': 'Label Encoder (clases)'
}

print(f"\n🔍 VERIFICANDO ARCHIVOS REQUERIDOS:")
all_files_present = True
for filename, description in required_files.items():
    file_path = Path('/content') / filename
    if file_path.exists():
        size_kb = file_path.stat().st_size / 1024
        print(f"   ✅ {description}: {filename} ({size_kb:.2f} KB)")
    else:
        print(f"   ❌ FALTA: {filename}")
        all_files_present = False

if not all_files_present:
    print(f"\n⚠️  ADVERTENCIA: Faltan archivos. Súbelos antes de continuar.")
    print(f"   Los archivos están en:")
    print(f"   • Entrega2/data/trained_models/randomforest_model.pkl")
    print(f"   • Entrega2/data/models/transformers/scaler.pkl")
    print(f"   • Entrega2/data/models/transformers/pca.pkl")
    print(f"   • Entrega2/data/models/transformers/label_encoder.pkl")
else:
    print(f"\n🤖 CARGANDO MODELO Y TRANSFORMADORES...")
    print("=" * 60)

    # Cargar Random Forest (más rápido que MLP)
    model = joblib.load('/content/randomforest_model.pkl')
    print("   ✅ Random Forest cargado (98.55% accuracy)")

    # Cargar transformadores
    scaler = joblib.load('/content/scaler.pkl')
    print("   ✅ Scaler cargado")

    pca = joblib.load('/content/pca.pkl')
    print(f"   ✅ PCA cargado ({pca.n_components_} componentes)")

    label_encoder = joblib.load('/content/label_encoder.pkl')
    print(f"   ✅ Label Encoder cargado ({len(label_encoder.classes_)} clases)")

    print("\n🏷️  ACTIVIDADES DETECTABLES:")
    for i, activity in enumerate(label_encoder.classes_):
        print(f"   {i+1}. {activity.replace('_', ' ').title()}")

    print("\n✅ MODELO LISTO PARA INFERENCIA\n")


📤 SUBIR ARCHIVOS DE MODELO

⚠️  IMPORTANTE: Sube estos 4 archivos en orden:
   1. randomforest_model.pkl
   2. scaler.pkl
   3. pca.pkl
   4. label_encoder.pkl

📁 Los archivos se guardarán en /content/




✅ Archivos subidos: 0

🔍 VERIFICANDO ARCHIVOS REQUERIDOS:
   ✅ Modelo Random Forest: randomforest_model.pkl (5307.24 KB)
   ✅ Scaler (normalización): scaler.pkl (2.53 KB)
   ✅ PCA (reducción dimensionalidad): pca.pkl (12.38 KB)
   ✅ Label Encoder (clases): label_encoder.pkl (0.53 KB)

🤖 CARGANDO MODELO Y TRANSFORMADORES...
   ✅ Random Forest cargado (98.55% accuracy)
   ✅ Scaler cargado
   ✅ PCA cargado (16 componentes)
   ✅ Label Encoder cargado (5 clases)

🏷️  ACTIVIDADES DETECTABLES:
   1. Caminar Hacia
   2. Caminar Regreso
   3. Girar
   4. Ponerse Pie
   5. Sentarse

✅ MODELO LISTO PARA INFERENCIA



In [3]:
# ============================================
# CONFIGURAR MEDIAPIPE POSE
# ============================================

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Configuración optimizada para tiempo real
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,  # Balance velocidad/precisión
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("✅ MediaPipe Pose configurado para tiempo real\n")

✅ MediaPipe Pose configurado para tiempo real



In [4]:
# ============================================
# FUNCIONES DE PROCESAMIENTO
# ============================================

def extract_landmarks(frame):
    """Extrae landmarks de MediaPipe y dibuja en el frame"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    annotated_frame = frame.copy()

    if results.pose_landmarks:
        # Dibujar landmarks
        mp_drawing.draw_landmarks(
            annotated_frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
        )

        # Extraer coordenadas
        landmarks = []
        for landmark in results.pose_landmarks.landmark:
            landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        return np.array(landmarks, dtype=np.float32), annotated_frame

    return None, annotated_frame


def compute_geometric_features(landmarks):
    """Calcula las 83 características geométricas del paper"""
    landmarks_reshaped = landmarks.reshape(33, 4)
    coords = landmarks_reshaped[:, :3]  # x, y, z

    # 1. DISTANCIAS (19 features)
    key_pairs = [
        (11, 12), (11, 13), (13, 15), (12, 14), (14, 16),
        (11, 23), (12, 24), (23, 24), (23, 25), (25, 27),
        (24, 26), (26, 28), (27, 29), (29, 31), (28, 30),
        (30, 32), (15, 17), (16, 18), (0, 1)
    ]

    distances = []
    for p1, p2 in key_pairs:
        dist = np.linalg.norm(coords[p1] - coords[p2])
        distances.append(dist)

    # 2. ÁNGULOS (15 features)
    def calculate_angle(a, b, c):
        ba = a - b
        bc = c - b
        cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        angle = np.arccos(np.clip(cosine, -1.0, 1.0))
        return np.degrees(angle)

    angles = [
        calculate_angle(coords[11], coords[13], coords[15]),  # Codo izq
        calculate_angle(coords[12], coords[14], coords[16]),  # Codo der
        calculate_angle(coords[13], coords[11], coords[23]),  # Hombro izq
        calculate_angle(coords[14], coords[12], coords[24]),  # Hombro der
        calculate_angle(coords[11], coords[23], coords[25]),  # Cadera izq
        calculate_angle(coords[12], coords[24], coords[26]),  # Cadera der
        calculate_angle(coords[23], coords[25], coords[27]),  # Rodilla izq
        calculate_angle(coords[24], coords[26], coords[28]),  # Rodilla der
        calculate_angle(coords[25], coords[27], coords[29]),  # Tobillo izq
        calculate_angle(coords[26], coords[28], coords[30]),  # Tobillo der
        calculate_angle(coords[11], coords[12], coords[24]),  # Torso superior
        calculate_angle(coords[23], coords[24], coords[26]),  # Torso inferior
        calculate_angle(coords[27], coords[23], coords[11]),  # Postura izq
        calculate_angle(coords[28], coords[24], coords[12]),  # Postura der
        calculate_angle(coords[15], coords[11], coords[12])   # Brazos
    ]

    # 3. RATIOS Y CARACTERÍSTICAS ADICIONALES (15 features)
    torso_height = np.linalg.norm(coords[11] - coords[23])
    leg_length_left = np.linalg.norm(coords[23] - coords[27])
    leg_length_right = np.linalg.norm(coords[24] - coords[28])
    arm_length_left = np.linalg.norm(coords[11] - coords[15])
    arm_length_right = np.linalg.norm(coords[12] - coords[16])

    ratios = [
        leg_length_left / (torso_height + 1e-6),
        leg_length_right / (torso_height + 1e-6),
        arm_length_left / (torso_height + 1e-6),
        arm_length_right / (torso_height + 1e-6),
        coords[23, 1] - coords[11, 1],  # Altura torso
        coords[27, 1] - coords[23, 1],  # Altura pierna izq
        coords[15, 0] - coords[11, 0],  # Extensión brazo izq
        coords[16, 0] - coords[12, 0],  # Extensión brazo der
        np.abs(coords[23, 0] - coords[24, 0]),  # Ancho caderas
        np.abs(coords[11, 0] - coords[12, 0]),  # Ancho hombros
        (coords[15, 1] + coords[16, 1]) / 2,    # Altura manos
        (coords[27, 1] + coords[28, 1]) / 2,    # Altura pies
        np.mean(coords[:, 1]),                  # Centro Y
        np.std(coords[:, 0]),                   # Dispersión X
        np.std(coords[:, 1])                    # Dispersión Y
    ]

    # Concatenar todas las features
    all_features = np.concatenate([landmarks, distances, angles, ratios])

    # Retornar exactamente 83 features (como en el entrenamiento)
    return all_features[:83].astype(np.float32)


def predict_activity(features):
    """Pipeline completo: Scaler → PCA → Random Forest"""
    # Reshape para sklearn
    features_reshaped = features.reshape(1, -1)

    # Normalizar
    features_scaled = scaler.transform(features_reshaped)

    # Reducir dimensionalidad
    features_pca = pca.transform(features_scaled)

    # Predecir
    prediction = model.predict(features_pca)[0]
    probabilities = model.predict_proba(features_pca)[0]

    # Decodificar
    predicted_class = label_encoder.inverse_transform([prediction])[0]
    confidence = probabilities.max()

    # Crear diccionario de probabilidades
    prob_dict = {
        label_encoder.classes_[i]: float(prob)
        for i, prob in enumerate(probabilities)
    }

    return predicted_class, confidence, prob_dict


print("✅ Funciones de procesamiento definidas\n")

✅ Funciones de procesamiento definidas



In [5]:
# ============================================
# FUNCIÓN PRINCIPAL DE GRADIO
# ============================================

def process_frame(image):
    """
    Función principal que procesa cada frame de la webcam

    Args:
        image: Frame de la webcam (numpy array RGB)

    Returns:
        annotated_image: Frame con landmarks dibujados
        activity_text: Texto con la actividad detectada
        probabilities: Diccionario de probabilidades por actividad
    """
    if image is None:
        return None, "⚠️ No hay imagen de la cámara", {}

    # Convertir RGB a BGR (OpenCV)
    frame_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extraer landmarks
    landmarks, annotated_frame = extract_landmarks(frame_bgr)

    if landmarks is None:
        # No se detectó pose
        annotated_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
        return annotated_rgb, "⚠️ No se detecta persona en el frame", {}

    # Calcular features geométricas
    features = compute_geometric_features(landmarks)

    # Predecir actividad
    predicted_class, confidence, prob_dict = predict_activity(features)

    # Formatear nombre de la actividad
    activity_name = predicted_class.replace('_', ' ').title()

    # Crear texto de resultado
    confidence_pct = confidence * 100

    # Emoji según confianza
    if confidence_pct >= 90:
        emoji = "✅"
    elif confidence_pct >= 75:
        emoji = "⚠️"
    else:
        emoji = "❓"

    result_text = f"""
🎯 **Actividad Detectada:** {activity_name}
{emoji} **Confianza:** {confidence_pct:.1f}%

📊 **Modelo:** Random Forest (98.55% accuracy)
    """

    # Dibujar resultado en el frame
    height, width = annotated_frame.shape[:2]

    # Fondo semitransparente
    overlay = annotated_frame.copy()
    cv2.rectangle(overlay, (10, 10), (width - 10, 120), (0, 0, 0), -1)
    annotated_frame = cv2.addWeighted(annotated_frame, 0.7, overlay, 0.3, 0)

    # Color según confianza
    if confidence_pct >= 90:
        color = (0, 255, 0)  # Verde
    elif confidence_pct >= 75:
        color = (0, 255, 255)  # Amarillo
    else:
        color = (0, 0, 255)  # Rojo

    # Texto en el frame
    cv2.putText(annotated_frame, f"Actividad: {activity_name}",
               (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    cv2.putText(annotated_frame, f"Confianza: {confidence_pct:.1f}%",
               (20, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Convertir BGR a RGB para Gradio
    annotated_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

    # Formatear probabilidades para display
    prob_display = {
        k.replace('_', ' ').title(): f"{v*100:.1f}%"
        for k, v in sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)
    }

    return annotated_rgb, result_text, prob_display


print("✅ Función principal de Gradio lista\n")

✅ Función principal de Gradio lista



In [6]:
# ============================================
# CREAR INTERFAZ GRADIO CON WEBCAM
# ============================================

print("🎨 CREANDO INTERFAZ GRADIO...\n")

# Crear interfaz
demo = gr.Interface(
    fn=process_frame,

    inputs=gr.Image(
        sources=["webcam"],  # Solo webcam
        type="numpy",
        label="📹 Cámara Web",
        streaming=True  # Modo streaming para tiempo real
    ),

    outputs=[
        gr.Image(
            type="numpy",
            label="🎥 Video Procesado (MediaPipe Landmarks)"
        ),
        gr.Markdown(
            label="🎯 Resultado de Clasificación"
        ),
        gr.JSON(
            label="📊 Probabilidades por Actividad"
        )
    ],

    title="🏃 Sistema de Clasificación de Actividades Humanas en Tiempo Real",

    description="""
    ### 📹 **Demo en Vivo - Universidad ICESI**

    **Modelo:** Random Forest (98.55% accuracy, 3.4s entrenamiento)

    **Actividades detectables:**
    - 🚶 Caminar Hacia la Cámara
    - 🚶‍♂️ Caminar de Regreso
    - 🔄 Girar
    - 🧍 Ponerse de Pie
    - 🪑 Sentarse

    **Instrucciones:**
    1. Permite acceso a tu cámara cuando el navegador lo solicite
    2. Colócate frente a la cámara (cuerpo completo visible)
    3. Realiza una de las actividades listadas
    4. El sistema clasificará en tiempo real

    **Pipeline:** MediaPipe (33 landmarks) → Feature Engineering (83 features) → Scaler → PCA (16 componentes) → Random Forest
    """,

    article="""
    ---
    **Desarrollado por:** Juan Ruiz & Tomás Quintero & Juan Quintero
    **Curso:** Inteligencia Artificial I - Universidad ICESI
    **Fecha:** Noviembre 2025
    **GitHub:** [video-ai-annotation-system](https://github.com/JRuiz1601/video-ai-annotation-system)
    """,

    live=True,  # Modo live para streaming
    cache_examples=False,
    allow_flagging="never"
)

print("✅ Interfaz Gradio creada\n")

🎨 CREANDO INTERFAZ GRADIO...

✅ Interfaz Gradio creada



In [7]:
# ============================================
# LANZAR APLICACIÓN
# ============================================

print("=" * 60)
print("🚀 LANZANDO APLICACIÓN GRADIO")
print("=" * 60)
print("\n📹 Accede a tu cámara cuando el navegador lo solicite")
print("🌐 Se generará una URL pública para compartir\n")

# Lanzar con share=True para URL pública
demo.launch(
    share=True,      # Genera URL pública (túnel ngrok)
    debug=False,     # Sin debug en producción
    show_error=True  # Mostrar errores si los hay
)

🚀 LANZANDO APLICACIÓN GRADIO

📹 Accede a tu cámara cuando el navegador lo solicite
🌐 Se generará una URL pública para compartir

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://70c35f5aeab00038ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
